<a href="https://colab.research.google.com/github/RomaniiEkaterina/Phyton-cases/blob/main/Logi_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-10-14 14:51:10--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-10-14 14:51:10--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.006s  

2025-10-14 14:51:11 (78.1

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

INFO  | 2022-11-26 00:01:03,024  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-26, количество людей с автопродлением подписки: 0

INFO  | 2022-11-26 00:01:03,027  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-27 00:01:02,455  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-27 00:01:02,480  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-27, количество людей с автопродлением подписки: 0

INFO  | 2022-11-27 00:01:02,483  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-28 00:01:02,406  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-28 00:01:02,424  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-28, количество людей с автопродлением подписки: 0

INFO  | 2022-11-28 00:01:02,427  | f

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:

def count_success_and_failure(file_path):
  with open(file_path,'r') as f:
    data=f.readlines()
  attemps=0
  errors=0

  for line in data:
    if 'Обновляем подписку пользователю id' in line:
      attemps+=1
    if 'ERROR' in line:
      errors+=1

  return(attemps-errors, errors)

count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [ ]:
import re
import statistics
from collections import defaultdict

def auto_renewal_sub(log_file_path):
    with open(log_file_path, 'r', encoding='utf-8') as f:
        data = f.readlines()

    totals = defaultdict(int)
    errors = defaultdict(int)

    daily_total_re = re.compile(
        r"\s+(\d{4}-\d{2}-\d{2}).*?количество людей с автопродлением подписки:\s*(\d+)",
        re.IGNORECASE
    )
    date_re = re.compile(r"(\d{4}-\d{2}-\d{2})")
    error_re = re.compile(r"\bERROR\b.*подпис", re.IGNORECASE)

    for line in data:
        m = daily_total_re.search(line)
        if m:
            date = m.group(1)
            count = int(m.group(2))
            if count > totals[date]:
                totals[date] = count
            continue


    dates_sorted = sorted(set(list(totals.keys()) ))
    values = [max(0, totals[d]) for d in dates_sorted]

    averages = []
    medians = []
    total = 0
    current = []

    for i, v in enumerate(values, start=1):
        total += v
        averages.append(total / i)
        current.append(v)
        medians.append(round(statistics.median(current)))


    with open("auto_renewal_sub.txt", "w", encoding="utf-8") as f:
        f.write("Среднее: " + str([round(x, 2) for x in averages]) + "\n")
        f.write("Медиана: " + str(medians) + "\n")

auto_renewal_sub("auto_purchase.log")




✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2025-10-14 14:51:11--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-10-14 14:51:11--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2025-10-14 14:5

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [ ]:

import re
from collections import defaultdict
from datetime import datetime

def sub_renewal_by_day(file_path):
    translate = {
        "Monday": "Понедельник", "Tuesday": "Вторник", "Wednesday": "Среда",
        "Thursday": "Четверг", "Friday": "Пятница", "Saturday": "Суббота",
        "Sunday": "Воскресенье",
    }
    ordered_days = ["Понедельник", "Вторник", "Среда",
                    "Четверг", "Пятница", "Суббота", "Воскресенье"]


    date_re   = re.compile(r"\b(\d{4}-\d{2}-\d{2})\b")
    error_re  = re.compile(r"^\s*ERROR\b", re.IGNORECASE)
    update_re = re.compile(r"^\s*INFO\b.*обнов\w*.*подпис\w*", re.IGNORECASE)

    total_updates = defaultdict(int)
    errors        = defaultdict(int)

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            m = date_re.search(line)
            if not m:
                continue
            d = datetime.strptime(m.group(1), "%Y-%m-%d")
            day = translate[d.strftime("%A")]

            if update_re.search(line):
                total_updates[day] += 1

            if error_re.search(line):
                errors[day] += 1


    success = {day: max(0, total_updates.get(day, 0) - errors.get(day, 0))
               for day in ordered_days}
    return success

success=sub_renewal_by_day('auto_purchase.log')

with open("weekdays.txt", "w", encoding="utf-8") as f:
    f.write("Количество обновлений подписки по дням недели:\n")
    for day, count in success.items():
        f.write(f"{day}: {count}\n")




✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2025-10-14 14:51:12--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2025-10-14 14:51:12--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt’

cor_weekdays.txt    100%[===================>]     238  --.-KB/s    in 0s      

2025-10-14 14:51:12 (5.37 MB/s) - ‘cor_

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
